## Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# Azure AI Agent with Code Interpreter Example

This notebook demonstrates using HostedCodeInterpreterTool with Azure AI Agents for Python code execution and mathematical problem solving.

## Features Covered:
- Setting up a code interpreter tool for agents
- Executing Python code through the agent
- Mathematical problem solving
- Accessing code interpreter inputs and outputs
- Working with streaming responses

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Project**: Access to an Azure AI Foundry project with deployed models
2. **Authentication**: Azure CLI installed and authenticated (`az login --use-device-code`)
3. **Environment Variables**: Set up your `.env` file with connection details
4. **Dependencies**: Required agent-framework packages installed

If you need to use a different tenant, specify the tenant ID:
```bash
az login --tenant <tenant-id>
```

## Import Libraries

Import the required libraries for Azure AI agent functionality.

In [3]:
import os
from pathlib import Path
from agent_framework import AgentRunResponse, ChatResponseUpdate, HostedCodeInterpreterTool
from agent_framework.azure import AzureAIAgentClient
from azure.ai.agents.models import (
    RunStepDeltaCodeInterpreterDetailItemObject,
)
from azure.identity.aio import AzureCliCredential

## Initial Setup

Initialize the Azure AI project client and set up authentication.

In [4]:
from pathlib import Path  # For working with file paths
import os  # For environment variables
from dotenv import load_dotenv  # For loading environment variables from .env file

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv('../../.env')  # Load environment variables from .env file

# Get configuration from environment variables
project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
model_deployment = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o")

# Validate and clean endpoint (remove any inline comments)
if project_endpoint and '#' in project_endpoint:
    print("⚠️  WARNING: Endpoint contains a '#' character - cleaning it...")
    project_endpoint = project_endpoint.split('#')[0].strip()

print(f"🔗 Project Endpoint: {project_endpoint}")
print(f"🤖 Model Deployment: {model_deployment}")

🔗 Project Endpoint: https://demopocaifoundry.services.ai.azure.com/api/projects/demoproject
🤖 Model Deployment: gpt-4o


## Helper Function for Code Interpreter Data

This helper function allows us to access and display the actual code that was executed by the code interpreter:

In [6]:
def print_code_interpreter_inputs(response: AgentRunResponse) -> None:
    """Helper method to access code interpreter data."""

    print("\nCode Interpreter Inputs during the run:")
    if response.raw_representation is None:
        return
    for chunk in response.raw_representation:
        if isinstance(chunk, ChatResponseUpdate) and isinstance(
            chunk.raw_representation, RunStepDeltaCodeInterpreterDetailItemObject
        ):
            print(chunk.raw_representation.input, end="")
    print("\n")

## Create and Run the Code Interpreter Agent

Now let's create an agent that can write and execute Python code to solve problems:

In [ ]:
async def main() -> None:
    """Example showing how to use the HostedCodeInterpreterTool with Azure AI."""
    print("=== Azure AI Agent with Code Interpreter Example ===")

    # For authentication, run `az login` command in terminal or replace AzureCliCredential with preferred
    # authentication option.
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=project_endpoint,
            model_deployment_name=model_deployment
        ) as chat_client,
    ):
        agent = chat_client.create_agent(
            name="CodingAgent",
            instructions=(
                "You are a helpful assistant that can write and execute Python code to solve problems."
            ),
            tools=HostedCodeInterpreterTool(),
        )
        query = "Generate the factorial of 100 using python code, show the code and execute it."
        print(f"User: {query}")
        response = await AgentRunResponse.from_agent_response_generator(agent.run_stream(query))
        print(f"Agent: {response}")
        # To review the code interpreter outputs, you can access
        # them from the response raw_representations, just uncomment the next line:
        # print_code_interpreter_inputs(response)

## Execute the Example

Run the main function to see the code interpreter in action:

In [8]:
# Run the main function
await main()

=== Azure AI Agent with Code Interpreter Example ===
User: Generate the factorial of 100 using python code, show the code and execute it.
Agent: Here's the Python code used to calculate the factorial of 100:

```python
import math

# Factorial of 100
factorial_100 = math.factorial(100)
print(factorial_100)
```

The factorial of 100 is:

93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000


## View Code Interpreter Inputs

Let's create another example that shows the actual code executed by the interpreter:

In [10]:
async def detailed_example() -> None:
    """Example that shows code interpreter inputs in detail."""
    print("=== Detailed Code Interpreter Example ===")

    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=project_endpoint,
            model_deployment_name=model_deployment
        ) as chat_client,
    ):
        agent = chat_client.create_agent(
            name="DetailedCodingAgent",
            instructions=(
                "You are a helpful assistant that can write and execute Python code to solve problems. "
                "Always explain your approach before writing code."
            ),
            tools=HostedCodeInterpreterTool(),
        )
        
        query = "Calculate the sum of squares of the first 20 numbers and create a simple visualization"
        print(f"User: {query}")
        response = await AgentRunResponse.from_agent_response_generator(agent.run_stream(query))
        print(f"Agent: {response}")
        
        # Show the actual code that was executed
        print_code_interpreter_inputs(response)

# Run the detailed example
await detailed_example()

=== Detailed Code Interpreter Example ===
User: Calculate the sum of squares of the first 20 numbers and create a simple visualization
Agent: To calculate the sum of squares of the first 20 numbers and create a visualization, here is the approach:

1. **Sum of Squares Calculation**:
   - The first 20 numbers are: 1, 2, 3, ..., 20.
   - Their squares are: \(1^2, 2^2, 3^2, ..., 20^2\).
   - Summing these values gives the desired result.

2. **Visualization**:
   - A bar chart or line plot can show the squares of these numbers, so it's easy to understand the relationship between the numbers and their squares.

I'll calculate the sum and produce the visualization. Let's proceed. The sum of squares of the first 20 numbers is **2870**.

The visualization above shows the squares of each number from 1 to 20 represented as bars. You can see how squares grow progressively larger with increasing values of the numbers.

Code Interpreter Inputs during the run:
import matplotlib.pyplot as plt

# Lis

## Try More Complex Examples

Let's test the agent with more complex programming tasks:

In [11]:
async def complex_examples():
    """Test the agent with more complex programming tasks."""
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=project_endpoint,
            model_deployment_name=model_deployment
        ) as chat_client,
    ):
        agent = chat_client.create_agent(
            name="AdvancedCodingAgent",
            instructions=(
                "You are an expert Python programmer. Write clean, efficient code and explain your solutions."
            ),
            tools=HostedCodeInterpreterTool(),
        )
        
        queries = [
            "Implement the Fibonacci sequence using recursion and iteration, compare their performance",
            "Create a simple data analysis of a randomly generated dataset with statistics and a plot",
            "Solve a quadratic equation ax² + bx + c = 0 for a=1, b=-5, c=6"
        ]
        
        for i, query in enumerate(queries, 1):
            print(f"\n--- Example {i}: {query} ---")
            response = await AgentRunResponse.from_agent_response_generator(agent.run_stream(query))
            print(f"Agent: {response}")
            print("\nCode executed:")
            print_code_interpreter_inputs(response)
            print("-" * 80)

# Run complex examples
await complex_examples()


--- Example 1: Implement the Fibonacci sequence using recursion and iteration, compare their performance ---
Agent: Sure! The Fibonacci sequence is a mathematical series where each number is the sum of the two preceding ones, typically starting with 0 and 1. The task involves creating two functions: one using recursion and another using iteration. We'll then compare their performance using a timing mechanism.

### Explanation
- **Recursive Solution**: This function will call itself to compute Fibonacci numbers. The base cases are `F(0) = 0` and `F(1) = 1`.
- **Iterative Solution**: This function will use a `for` loop to calculate Fibonacci numbers, making it more efficient because recursion incurs a function-call overhead.

I will implement both solutions and compare their performance for a given Fibonacci number \( n \).For \( n = 30 \), here are the results of the Fibonacci calculations:

1. **Recursive Implementation**:
   - Result: \( 832,040 \)
   - Time taken: \( 0.1249 \, \text

## Key Takeaways

1. **Code Interpreter Tool**: The `HostedCodeInterpreterTool` allows agents to execute Python code in a secure environment
2. **Mathematical Computation**: Perfect for complex calculations like factorials, statistics, and mathematical operations
3. **Data Analysis**: Can perform data analysis tasks including generating plots and statistics
4. **Code Visibility**: You can access the actual code executed through `print_code_interpreter_inputs()`
5. **Streaming Support**: Works with both streaming and non-streaming responses
6. **Security**: Code execution happens in a secure, isolated environment
7. **Versatility**: Can handle various programming tasks from simple calculations to complex algorithms

## Best Practices

1. **Clear Instructions**: Provide clear instructions about what kind of code execution you need
2. **Error Handling**: The agent will handle code errors gracefully and suggest fixes
3. **Resource Management**: Use async context managers for proper cleanup
4. **Code Review**: Always review the executed code for understanding and validation
5. **Performance**: Be mindful of computational complexity for large-scale operations

## Use Cases

- Mathematical problem solving
- Data analysis and visualization
- Algorithm implementation and testing
- Educational coding assistance
- Prototype development
- Statistical computations